In [ ]:
!apt-get install -yq jq > /dev/null
!jq --version

The system cannot find the path specified.


In [ ]:
%%capture [--no-stdout]
!pip install fhir-aggregator-client --no-cache-dir --quiet

In [1]:
%%capture [--no-stdout]
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt

opt.classes="display nowrap compact"
opt.buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
opt.maxBytes=0
%env FHIR_BASE= https://google-fhir.fhir-aggregator.org

!fq vocabulary vocabulary.tsv --fhir-base-url $FHIR_BASE
df = pd.read_csv('vocabulary.tsv', sep='\t').fillna('')


# Vocabulary filtering examples

It's easy enough to get a straightforward table response - just utilize Pandas to filter appropriately to your interest. Some examples below:

# Exploring a study
Finding the vocabulary specific study is easy - filter according to the desired identifier in the `research_study_identifiers` column.

In [3]:
scratch_df = df[df['research_study_identifiers'].str.contains('1KG')]
scratch_df

research_study_identifiers                        path  \
3740                        1KG     ServiceRequest.category   
3741                        1KG         ServiceRequest.code   
3742                        1KG      DocumentReference.type   
3743                        1KG      DocumentReference.type   
3744                        1KG  DocumentReference.category   
...                         ...                         ...   
3821                        1KG           Patient.extension   
3822                        1KG           Patient.extension   
3823                        1KG           Patient.extension   
3824                        1KG           Patient.extension   
3825                        1KG           Patient.extension   

                                          documentation       code  \
3740  https://hl7.org/fhir/R4B/servicerequest-defini...  108252007   
3741  https://hl7.org/fhir/R4B/servicerequest-defini...   15220000   
3742  https://hl7.org/fhir/R4B/documentreference-def...        VCF   
3743  https://hl7.org/fhir/R4B/documentreference-def...        NEW   
3744  https://hl7.org/fhir/R4B/documentreference-def...          1   
...                                                 ...        ...   
3821  https://hl7.org/fhir/R4B/patient-definitions.h...        LWK   
3822  https://hl7.org/fhir/R4B/patient-definitions.h...        ASW   
3823  https://hl7.org/fhir/R4B/patient-definitions.h...        MXL   
3824  https://hl7.org/fhir/R4B/patient-definitions.h...        TSI   
3825  https://hl7.org/fhir/R4B/patient-definitions.h...        GIH   

                   display                                         system  \
3740  Laboratory procedure                         http://snomed.info/sct   
3741       Laboratory test                         http://snomed.info/sct   
3742                   VCF  https://ftp.1000genomes.ebi.ac.uk/data_format   
3743                   NEW  https://ftp.1000genomes.ebi.ac.uk/data_format   
3744          Chromosome 1   https://ftp.1000genomes.ebi.ac.uk/chromosome   
...                    ...                                            ...   
3821                   LWK                                                  
3822                   ASW                                                  
3823                   MXL                                                  
3824                   TSI                                                  
3825                   GIH                                                  

                                          extension_url  count low high  \
3740                                                       1.0            
3741                                                       1.0            
3742                                                      46.0            
3743                                                       2.0            
3744                                                       2.0            
...                                                 ...    ...  ..  ...   
3821  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  116.0            
3822  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  112.0            
3823  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  107.0            
3824  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  112.0            
3825  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  113.0            

                                                    url  \
3740  https://google-fhir.fhir-aggregator.org/Servic...   
3741  https://google-fhir.fhir-aggregator.org/Servic...   
3742  https://google-fhir.fhir-aggregator.org/Docume...   
3743  https://google-fhir.fhir-aggregator.org/Docume...   
3744  https://google-fhir.fhir-aggregator.org/Docume...   
...                                                 ...   
3821  https://google-fhir.fhir-aggregator.org/Patien...   
3822  https://google-fhir.fhir-aggregator.org/Patien...   
3823  https://google-fhir.fhir-aggregator.org/Pati

# Exploring available conditions
Or query the vocab table for the available condition codes of the various patients in each research study.

In [ ]:
scratch_df = df[df['path'].str.contains('Condition.code')]
scratch_df

# Something more elaborate
In lieu of pandas filtering, we can use curl to perform more elaborate filtering. In this case we're searching for a specific condition, Neuroblastoma. We'll leverage the `jq` command line utility for the purposes of formatting the output appropriately. 
 
Here's what each part of the below code is doing:
1. FHIR Query: The curl command sends the FHIR query to the server.
2. Data Extraction and Formatting: jq extracts the system, code, and display information from the coding elements within each Condition resource and formats them as TSV.
3. Sorting: The output is piped to sort to arrange the entries alphabetically.
4. Deduplication: The -u option in sort removes any duplicate entries, leaving only unique combinations of system, code, and display.
5. Output: The final result is a sorted and deduplicated list of coding information for conditions starting with "Adenocarcinoma," presented as TSV.

In [ ]:
!curl -s $FHIR_BASE'/Condition?code:text=Neuroblastoma&_count=1000&_total=accurate&_elements=subject,extension,code'  | jq -rc ' .entry[] | .resource | .code.coding[] | [.system, .code, .display] | @tsv' | sort | uniq -c
#| jq -rc '.entry[] | .resource | [.subject.reference, (.extension[] | .valueReference.reference)] '